In [2]:
#imports
import requests
import json
import pandas as pd
import numpy as np
import warnings
import random
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
from datetime import datetime
import time

In [3]:
# Getting MongoDB data ( JSON )
Aircon_Data = pd.read_json("data/SPGG_aircon_status.json", convert_dates=False)
Sensors_Data = pd.read_json("data/SPGG_readings.json", convert_dates=False)

# Normalize the data
Aircon_rows = []

for _, row in Aircon_Data.iterrows():
    date = row['date']
    time = row['time']
    
    flattened_row = {
        "date": date,
        "time": time
    }
    
    fc_readings = row['FC_FullStatus_Readings']
    
    for unit, data in fc_readings.items():
        flattened_row[f"{unit}_Status"] = data.get("Status", None)
        flattened_row[f"{unit}_Fan_Status"] = data.get("Fan_Status", None)
        flattened_row[f"{unit}_Set_Point"] = data.get("Set_Point", None)
        flattened_row[f"{unit}_Operation_Mode"] = data.get("Operation_Mode", None)
    
    Aircon_rows.append(flattened_row)

Sensors_rows = []
for _, row in Sensors_Data.iterrows():
    invalid_input = False
    
    date = row['date']
    time = row['time']
    
    flattened_row = {
        "date": date,
        "time": time
    }
    
    
    lorawan_readings = row['Lorawan_Readings']
    
    if isinstance(lorawan_readings, dict):
        for unit, data in lorawan_readings.items():
            if isinstance(data, dict):  # Ensure that each item in Lorawan_Readings is a dictionary
                for key, value in data.items():
                    flattened_row[f"{unit}_{key}"] = value
            
    energy_readings = row['Energy_Readings']
    total_power = 0
    total_energy = 0
    invalid_input_power = False
    invalid_input_energy = False
    
    for unit, data in energy_readings.items():
        power = data.get('Power', None)
        energy = data.get('Energy', None)
        if power is None:
            invalid_input_power = True
        if energy is None:
            invalid_input_energy = True
        total_power += power
        total_energy += energy
        
    if invalid_input_power:
        total_power = None
    if invalid_input_energy:
        total_energy = None
        
    flattened_row["Total_Energy"] = total_energy
    flattened_row["Total_Power"] = total_power
    
    Sensors_rows.append(flattened_row)

    
Aircon_Normalize_Data = pd.DataFrame(Aircon_rows)
Sensors_Normalize_Data = pd.DataFrame(Sensors_rows)

Aircon_Normalize_Data['datetime_str'] = Aircon_Normalize_Data['date'] + ' ' + Aircon_Normalize_Data['time']
Aircon_Normalize_Data['datetime'] = Aircon_Normalize_Data['datetime_str'].apply(lambda x: datetime.strptime(x, "%a %b %d %Y %I:%M:%S %p"))
Aircon_Normalize_Data['timestamp'] = Aircon_Normalize_Data['datetime'].apply(lambda x: int(x.timestamp()))

Sensors_Normalize_Data['datetime_str'] = Sensors_Normalize_Data['date'] + ' ' + Sensors_Normalize_Data['time']
Sensors_Normalize_Data['datetime'] = Sensors_Normalize_Data['datetime_str'].apply(lambda x: datetime.strptime(x, "%a %b %d %Y %I:%M:%S %p"))
Sensors_Normalize_Data['timestamp'] = Sensors_Normalize_Data['datetime'].apply(lambda x: int(x.timestamp()))

merged_data = pd.merge_asof(
    Aircon_Normalize_Data,  # Left DataFrame
    Sensors_Normalize_Data,      # Right DataFrame
    on='timestamp',   # Key column
    direction='nearest'    # Match the nearest time
)

temperature_col = [
    col for col in merged_data.columns 
    if "temperature" in col.lower() and "24e124136d336145" not in col.lower()
]

outdoor_col = [
    col for col in merged_data.columns 
    if "24e124136d336145" in col.lower()
]

door_col = [
    col for col in merged_data.columns
    if "24e124141e173857" in col.lower()
]

humidity_col = [
    col for col in merged_data.columns 
    if "humidity" in col.lower()
]

occupancy_col_total_in = [
    col for col in merged_data.columns
    if "line_1_total_in" in col.lower()
]

occupancy_col_total_out = [
    col for col in merged_data.columns
    if "line_1_total_out" in col.lower()
]

def get_unit_columns(unit_number, columns):
    return [col for col in columns if f"FC_Unit_{unit_number}" in col]

aircon_units = len([
    col for col in merged_data.columns
    if "FC_Unit_" in col and "_Status" in col and "Fan" not in col
])

aircon_units_cols = {}

for unit in range(1, aircon_units + 1):
    aircon_units_cols[f'Unit_{unit}'] = get_unit_columns(unit, merged_data.columns)
    

# Creating the wanted fields
final_data = pd.DataFrame()
final_data["timestamp"] = merged_data["timestamp"]

final_data["temperature"] = merged_data[temperature_col].apply(lambda x: round(x.mean(), 3), axis=1)
final_data["humidity"] = merged_data[humidity_col].apply(lambda x: round(x.mean(),3), axis=1)

final_data['power_consumption'] = merged_data['Total_Power']
final_data['energy_consumption'] = merged_data['Total_Energy']

final_data["outdoor_temperature"] = merged_data[outdoor_col]['24E124136D336145_temperature'].ffill()
final_data["outdoor_humidity"] = merged_data[outdoor_col]['24E124136D336145_humidity'].ffill()

#0 is door closed, 1 is door opened
final_data["door_status"] = merged_data[door_col]["24E124141E173857_magnet_status"].ffill()

merged_data[occupancy_col_total_in] = merged_data[occupancy_col_total_in].fillna(method='bfill')
merged_data[occupancy_col_total_out] = merged_data[occupancy_col_total_out].fillna(method='bfill')

final_data['occupancy'] = (
    merged_data[occupancy_col_total_in].sum(axis=1) - merged_data[occupancy_col_total_out].sum(axis=1)
).clip(lower=0)

for unit, columns in aircon_units_cols.items():
    for column in columns:
        if 'set_point' in column:
            final_data[column] = merged_data[column].replace(0, pd.NA).ffill()
        else:
            final_data[column] = merged_data[column].replace("ERROR", pd.NA).ffill()

final_data.dropna(inplace=True)
# final_data.reset_index(drop=True, inplace=True)#######################################


final_data.to_csv('final_data.csv', index=False)

#print(final_data.columns)

#print(final_data.head())

def getFCData(data, row_index):
    settings = []
    for i in range(1, aircon_units + 1):
        settings.append(data[f"FC_Unit_{i}_Status"].iloc[row_index])
        settings.append(data[f"FC_Unit_{i}_Fan_Status"].iloc[row_index])
        settings.append(data[f"FC_Unit_{i}_Set_Point"].iloc[row_index])
        settings.append(data[f"FC_Unit_{i}_Operation_Mode"].iloc[row_index])
        
    return settings

def is_same_settings(data, curr_row_index, next_row_index):   
    return True if (getFCData(data, curr_row_index) == getFCData(data, next_row_index)) else False

# def is_same_door_status(data, curr_row_index, next_row_index):
#     return True if (data['door_status'][curr_row_index] == data['door_status'][next_row_index]) else False



def is_all_off(data, curr_row_index, check_for_off):
    for i in range(1, aircon_units + 1):
        if data[f"FC_Unit_{i}_Status"].iloc[curr_row_index] == "ON":
            return not check_for_off
        
    return check_for_off


# Get the time taken and energy consumed to get to the *target temp
# 0, 1, 2, 3
aircon_status_result = pd.DataFrame()
#aircon_status_result = pd.DataFrame(columns=['time_taken', 
#                                              'energy_consumption', 
#                                              'previous_temp', 
#                                              'previous_humi',
#                                              'current_temp',
#                                              'current_humi'])

total_final_rows = final_data.shape[0]
Aircon_Normalize_Data = Aircon_Normalize_Data.drop(['date', 'time', 'datetime_str', 'datetime', 'timestamp'], axis=1)
for i in range(total_final_rows - 1, -1, -1):
    
    rows = []
    time_taken = []
    energy_consumption = []
    previous_temp = []
    previous_humi = []
    
    curr_timestamp = final_data["timestamp"].iloc[i]
    curr_energy = final_data["energy_consumption"].iloc[i]
    curr_temperature = final_data["temperature"].iloc[i]
    curr_humidity = final_data["humidity"].iloc[i]

    
    while i >= 0 and is_same_settings(final_data, i - 1, i): #and is_same_door_status(final_data, i - 1,i):#############
        rows.append(i - 1)
        time_taken.append(curr_timestamp - final_data["timestamp"].iloc[i - 1])
        energy_consumption.append(curr_energy - final_data["energy_consumption"].iloc[i - 1])
        previous_temp.append(final_data["temperature"].iloc[i - 1])
        previous_humi.append(final_data["humidity"].iloc[i - 1])
        
        i -= 1
        
    temp_df = pd.DataFrame({
            'timestamp': [curr_timestamp],
            'rows': [rows],
            'time_taken': [time_taken],
            'energy_consumption': [energy_consumption],
            'previous_temp': [previous_temp],
            'previous_humi': [previous_humi],
            'outdoor_temp': [final_data['outdoor_temperature'].iloc[i]],
            'outdoor_humi': [final_data['outdoor_humidity'].iloc[i]],
            'current_temp': [curr_temperature],
            'current_humi': [curr_humidity],
            'door_status': [final_data["door_status"].iloc[i]]
        })
    for col in Aircon_Normalize_Data.columns:
        temp_df[col] = final_data[col].iloc[i]
    
        
    aircon_status_result = pd.concat([aircon_status_result, temp_df], ignore_index=False)
        
        
        
        
        
print("Finished")
aircon_status_result = aircon_status_result.sort_values(by=['current_temp'], ascending=False)
aircon_status_result.to_csv('aircon_status.csv', index=False)
aircon_status_result.info()




C:\Users\yy\AppData\Local\Temp\ipykernel_15028\3252054830.py:153: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_data[occupancy_col_total_in] = merged_data[occupancy_col_total_in].fillna(method='bfill')
C:\Users\yy\AppData\Local\Temp\ipykernel_15028\3252054830.py:154: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_data[occupancy_col_total_out] = merged_data[occupancy_col_total_out].fillna(method='bfill')


Finished
<class 'pandas.core.frame.DataFrame'>
Index: 2553 entries, 0 to 0
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   timestamp                 2553 non-null   int64  
 1   rows                      2553 non-null   object 
 2   time_taken                2553 non-null   object 
 3   energy_consumption        2553 non-null   object 
 4   previous_temp             2553 non-null   object 
 5   previous_humi             2553 non-null   object 
 6   outdoor_temp              2553 non-null   float64
 7   outdoor_humi              2553 non-null   float64
 8   current_temp              2553 non-null   float64
 9   current_humi              2553 non-null   float64
 10  door_status               2553 non-null   float64
 11  FC_Unit_1_Status          2553 non-null   object 
 12  FC_Unit_1_Fan_Status      2553 non-null   object 
 13  FC_Unit_1_Set_Point       2553 non-null   float64
 14  FC_Unit

In [5]:
target_temp_range = range(20, 30)
time_factor = 0.5
energy_factor = 0.5
acceptable_range = 0.8
total_rows = aircon_status_result.shape[0]
stored_dictionary = {}

aircon_status_result.info()


def getRowData(row_index):
    temperature = aircon_status_result["current_temp"].iloc[row_index]
    humidity = aircon_status_result["current_humi"].iloc[row_index]
    
    return [temperature, humidity]

def getArrayData(row_index, array_index):
    time_taken = aircon_status_result["time_taken"].iloc[row_index]
    energy_consumption = aircon_status_result["energy_consumption"].iloc[row_index]
    temperature = aircon_status_result["previous_temp"].iloc[row_index]
    humidity = aircon_status_result["previous_humi"].iloc[row_index]
    
    return [temperature[array_index], humidity[array_index], time_taken[array_index], energy_consumption[array_index]]

def comparePath(best_path, current_path):    
    if best_path['factor'] > current_path['factor']:
        return True
    
    return False

for target_temp in target_temp_range:
    paths = {}
    valid_target_found = False  # Flag to track if a valid target temperature is achieved

    def findBestCombi(current_row_index):
        global paths
        curr_temperature, curr_humidity = getRowData(current_row_index)
        
        if is_all_off(aircon_status_result, current_row_index, True):
            paths[current_row_index] = {
                'energy_consumption': [], 
                'starting_temp': curr_temperature, 
                'starting_humi': curr_humidity, 
                'time_taken': [], 
                'factor': float('inf'), 
                'path': []
            }
            return paths[current_row_index]
        
        if current_row_index in paths:
            return paths[current_row_index]
        
        for i in range(len(aircon_status_result['rows'].iloc[current_row_index])):
            array_data = getArrayData(current_row_index, i)
            if abs(array_data[0] - target_temp) < acceptable_range:
                print(f"Target {target_temp} found at row {current_row_index}")
                valid_target_found = True
                curr_path = {
                    'energy_consumption': [array_data[3]],
                    'time_taken': [array_data[2]],
                    'factor': array_data[3] * energy_factor + array_data[2] * time_factor,
                    'starting_temp': curr_temperature,
                    'starting_humi': curr_humidity,
                    'ending_temp': array_data[0],
                    'ending_humi': array_data[1],
                    'path': [current_row_index]
                }
                paths[current_row_index] = curr_path
                return paths[current_row_index]

        for i in range(len(aircon_status_result['rows'].iloc[current_row_index])):
            previous_data = getArrayData(current_row_index, i)
            for j in range(current_row_index + 1, total_rows):
                next_data = getRowData(j)
                if abs(previous_data[0] - next_data[0]) < acceptable_range:
                    path = findBestCombi(j)
                    if path and path['energy_consumption']:
                        curr_path = {
                            'energy_consumption': [previous_data[3]] + path['energy_consumption'],
                            'time_taken': [previous_data[2]] + path['time_taken'],
                            'starting_temp': previous_data[0],
                            'starting_humi': previous_data[1],
                            'ending_temp': path['ending_temp'],
                            'ending_humi': path['ending_humi'],
                            'path': [current_row_index] + path['path']
                        }
                        curr_path['factor'] = sum(curr_path['energy_consumption']) * energy_factor + sum(curr_path['time_taken']) * time_factor
                        if current_row_index in paths:
                            if comparePath(paths[current_row_index], curr_path):
                                paths[current_row_index] = curr_path
                        else:
                            paths[current_row_index] = curr_path
        
        if current_row_index not in paths:
            paths[current_row_index] = {
                'energy_consumption': [], 
                'starting_temp': curr_temperature, 
                'starting_humi': curr_humidity, 
                'time_taken': [], 
                'factor': float('inf'), 
                'path': []}
        
        return paths[current_row_index]
    
    for i in range(total_rows):
        if i not in paths:
            for j in range(len(aircon_status_result['rows'].iloc[i])):
                array_data = getArrayData(i, j)
                path = findBestCombi(i)
                if path and path['energy_consumption']:
                    curr_path = {
                        'energy_consumption': [array_data[3]] + path['energy_consumption'],
                        'time_taken': [array_data[2]] + path['time_taken'],
                        'starting_temp': array_data[0],
                        'starting_humi': array_data[1],
                        'ending_temp': path['ending_temp'],
                        'ending_humi': path['ending_humi'],
                        'path': [i] + path['path']
                    }
                    curr_path['factor'] = sum(curr_path['energy_consumption']) * energy_factor + sum(curr_path['time_taken']) * time_factor
                    if i in paths:
                        if comparePath(paths[i], curr_path):
                            paths[i] = curr_path
                    else:
                        paths[i] = curr_path
                else:
                    paths[i] = {'energy_consumption': [], 'starting_temp': array_data[0], 'starting_humi': array_data[1], 'time_taken': [], 'factor': float('inf'), 'path': []}
    
    if not any(path['factor'] < float('inf') for path in paths.values()):
        print(f"Target temp {target_temp} not achievable.")
        continue  # Move to the next target temperature
    else:
        stored_dictionary[f"Dictionary for target temp: {target_temp}"] = paths.copy()

print("Finished")


<class 'pandas.core.frame.DataFrame'>
Index: 2553 entries, 0 to 0
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   timestamp                 2553 non-null   int64  
 1   rows                      2553 non-null   object 
 2   time_taken                2553 non-null   object 
 3   energy_consumption        2553 non-null   object 
 4   previous_temp             2553 non-null   object 
 5   previous_humi             2553 non-null   object 
 6   outdoor_temp              2553 non-null   float64
 7   outdoor_humi              2553 non-null   float64
 8   current_temp              2553 non-null   float64
 9   current_humi              2553 non-null   float64
 10  door_status               2553 non-null   float64
 11  FC_Unit_1_Status          2553 non-null   object 
 12  FC_Unit_1_Fan_Status      2553 non-null   object 
 13  FC_Unit_1_Set_Point       2553 non-null   float64
 14  FC_Unit_1_Operat

In [30]:
#asssuming there's multiple key for target temp
unique_keys = [key for key in stored_dictionary.keys() if key.startswith("Dictionary for target temp") or key.startswith("Best alternative for target temp")]
print(unique_keys)

current_temperature = 25
target_temp = 20
key_to_find = f"Dictionary for target temp: {target_temp}"

while key_to_find not in stored_dictionary:
    if (target_temp - current_temperature) < 0: #value is negative -> temp is going from higher to lower\
        key_to_find = f"Dictionary for target temp: {target_temp + 1}"

    else:# value is positive -> temp is doing lower to higher temp
        key_to_find = f"Dictionary for target temp: {target_temp - 1}"


def expandPath(row_index):
    for i in range(1, aircon_units + 1):
        print("UNIT " + str(i) + ":", end=' ')
        print(aircon_status_result[f"FC_Unit_{i}_Status"].iloc[row_index], end=' ')
        print(aircon_status_result[f"FC_Unit_{i}_Fan_Status"].iloc[row_index], end=' ')
        print(aircon_status_result[f"FC_Unit_{i}_Set_Point"].iloc[row_index], end=' ')
        print(aircon_status_result[f"FC_Unit_{i}_Operation_Mode"].iloc[row_index], end=' ')
        print("")
        
def findClosestTemperature(current_temp, paths):
    """Find the index of the closest available temperature in the paths dictionary."""
    closest_temp_index = None
    smallest_difference = float('inf')
    
    for key, value in paths.items():
        if value['starting_temp'] and value['path']:  #ensure that paths[index] has both values in 'starting_temp' and 'path'
            difference = abs(value['starting_temp'] - current_temp)
            if difference < smallest_difference:
                smallest_difference = difference
                closest_temp_index = key   
    return closest_temp_index


# Check if either key exists in stored_dictionary
if key_to_find in stored_dictionary:
    stored_dict_key = stored_dictionary[key_to_find]

    filtered_paths = {
        key: value for key, value in stored_dict_key.items()
        if (abs(value['starting_temp'] - current_temperature) < acceptable_range)
    }
    
    if filtered_paths:
        # Find the path with the smallest factor
        smallest_factor_path = min(filtered_paths.keys(), key=lambda x: filtered_paths[x]['factor'])
        print(stored_dict_key[smallest_factor_path])
    
        for index, value in enumerate(stored_dict_key[smallest_factor_path]['path']):
            expandPath(value)
            print("For", end=" ")
            hours, remainder = divmod(stored_dict_key[smallest_factor_path]['time_taken'][index], 3600)
            minutes, seconds = divmod(remainder, 60)
            print(f"Hours: {hours}, Minutes: {minutes}, Seconds: {seconds}")
            print("")
    else:
        print("No paths found within the acceptable range.")
        print("Finding the closest temperature in the algorithm.")
    
        closest_temp_index = findClosestTemperature(current_temperature, paths)
        
        if closest_temp_index is not None:
            
            print("Closest temperature found at index:", closest_temp_index , "\n")
            print(paths[closest_temp_index])
            
            for index, value in enumerate(stored_dict_key[closest_temp_index]['path']):
                expandPath(value)
                print("For", end=" ")
                hours, remainder = divmod(stored_dict_key[closest_temp_index]['time_taken'][index], 3600)
                minutes, seconds = divmod(remainder, 60)
                print(f"Hours: {hours}, Minutes: {minutes}, Seconds: {seconds}")
                print("")
        else:
            print("No valid paths available, even for the closest temperature.")

else:
    print(f"{key_to_find} does not exist")





['Dictionary for target temp: 21', 'Dictionary for target temp: 22', 'Dictionary for target temp: 23', 'Dictionary for target temp: 24', 'Dictionary for target temp: 25', 'Dictionary for target temp: 26', 'Dictionary for target temp: 27', 'Dictionary for target temp: 28', 'Dictionary for target temp: 29']
{'energy_consumption': [0.4899999999997817, 0.08999999999923602, 0.6100000000005821], 'time_taken': [628, 65, 485], 'starting_temp': 24.85, 'starting_humi': 73.286, 'ending_temp': 21.5, 'ending_humi': 80.0, 'path': [2458, 2483, 2503], 'factor': 589.5949999999998}
UNIT 1: OFF MED 22.0 COOL 
UNIT 2: OFF MED 22.0 COOL 
UNIT 3: ON MED 22.0 COOL 
UNIT 4: ON MED 22.0 COOL 
UNIT 5: ON MED 22.0 COOL 
UNIT 6: ON MED 22.0 COOL 
For Hours: 0, Minutes: 10, Seconds: 28

UNIT 1: ON LOW 22.0 COOL 
UNIT 2: ON LOW 22.0 COOL 
UNIT 3: ON LOW 22.5 COOL 
UNIT 4: ON LOW 22.5 COOL 
UNIT 5: ON LOW 22.5 AUTO 
UNIT 6: ON LOW 22.5 COOL 
For Hours: 0, Minutes: 1, Seconds: 5

UNIT 1: ON LOW 22.0 COOL 
UNIT 2: ON 